<a href="https://colab.research.google.com/github/t-aridi/DS2002-002-Assignments/blob/main/Finance%20API%20HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
import pandas as pd
import json
from typing import List, Dict, Any

class YahooFinanceClient:
    """
    A client to interact with Yahoo Finance APIs.
    """
    BASE_URL_QUOTE = "https://yfapi.net/v6/finance/quote"
    BASE_URL_QUOTE_SUMMARY = "https://yfapi.net/v11/finance/quoteSummary/{symbol}"
    BASE_URL_TRENDING = "https://yfapi.net/v1/finance/trending/US"

    def __init__(self, api_key: str) -> None:
        self.api_key = api_key
        self.headers = {'x-api-key': self.api_key}
        self.session = requests.Session()

    def fetch_stock_data(self, symbols: str) -> List[Dict[str, Any]]:
        """
        Fetch basic stock data for one or more symbols.
        """
        params = {"symbols": symbols}
        response = self.session.get(self.BASE_URL_QUOTE, headers=self.headers, params=params)
        data = response.json()
        print("Stock Data Response:", json.dumps(data, indent=2))

        stock_data: List[Dict[str, Any]] = []
        results = data.get('quoteResponse', {}).get('result', [])
        if results:
            for stock in results:
                stock_data.append({
                    'Stock Ticker': stock.get('symbol', 'N/A'),
                    'Company': stock.get('longName', 'N/A'),
                    'Current Market Price': stock.get('regularMarketPrice', 'N/A')
                })
        else:
            print("Error: No stock data found.")
        return stock_data

    def fetch_additional_data(self, symbol: str, module: str) -> Dict[str, Any]:
        """
        Fetch additional stock data from a given module.
        """
        url = self.BASE_URL_QUOTE_SUMMARY.format(symbol=symbol)
        params = {"modules": module}
        response = self.session.get(url, headers=self.headers, params=params)
        data = response.json()
        print("Additional Data Response:", json.dumps(data, indent=2))

        results = data.get('quoteSummary', {}).get('result')
        if results and isinstance(results, list) and results:
            return results[0].get(module, {})
        else:
            print("Error: Could not retrieve additional data.")
            return {}

    def fetch_trending_stocks(self) -> List[Dict[str, Any]]:
        """
        Fetch currently trending stocks in the US.
        """
        response = self.session.get(self.BASE_URL_TRENDING, headers=self.headers)
        data = response.json()
        print("Trending Stocks Response:", json.dumps(data, indent=2))

        trending_stocks: List[Dict[str, Any]] = []
        results = data.get('finance', {}).get('result')
        if results and isinstance(results, list) and results:
            quotes = results[0].get('quotes', [])
            for stock in quotes:
                trending_stocks.append({
                    'Stock Ticker': stock.get('symbol', 'N/A'),
                    'Company': stock.get('shortName', stock.get('longName', 'N/A')),
                    'Current Market Price': stock.get('regularMarketPrice', 'N/A'),
                    '52 Week High': stock.get('fiftyTwoWeekHigh', 'N/A'),
                    '52 Week Low': stock.get('fiftyTwoWeekLow', 'N/A')
                })
        else:
            print("Error: No trending stocks data found.")
        return trending_stocks

def main():
    # Prompt the user for the API key and create a client instance.
    api_key = input("Enter your API key: ").strip()
    client = YahooFinanceClient(api_key)

    # Fetch basic stock data.
    symbols = input("Enter stock symbols separated by commas: ").strip()
    stock_data = client.fetch_stock_data(symbols)
    if stock_data:
        for stock in stock_data:
            price = stock['Current Market Price']
            # Format the price nicely if it is a number.
            price_display = f"${price:.2f}" if isinstance(price, (int, float)) else price
            print(f"Stock Ticker: {stock['Stock Ticker']}, Company: {stock['Company']}, Current Market Price: {price_display}")

    # Fetch additional stock data.
    symbol = input("Enter a stock symbol for additional data: ").strip()
    module = input("Choose a module (summaryDetail, financialData, defaultKeyStatistics): ").strip()
    additional_data = client.fetch_additional_data(symbol, module)
    if additional_data:
        df = pd.DataFrame([additional_data])
        print(df)

    # Fetch trending stocks data.
    trending_stocks = client.fetch_trending_stocks()
    if trending_stocks:
        df = pd.DataFrame(trending_stocks)
        print(df)

if __name__ == "__main__":
    main()


Enter your API key: HW1PtNJlcNaKXD3AXIgks5uXBaqHWsN43kOVdDje
Enter stock symbols separated by commas: BAH
Stock Data Response: {
  "quoteResponse": {
    "result": [
      {
        "language": "en-US",
        "region": "US",
        "quoteType": "EQUITY",
        "typeDisp": "Equity",
        "quoteSourceName": "Delayed Quote",
        "triggerable": true,
        "customPriceAlertConfidence": "HIGH",
        "shortName": "Booz Allen Hamilton Holding Cor",
        "longName": "Booz Allen Hamilton Holding Corporation",
        "regularMarketPrice": 113.49,
        "currency": "USD",
        "regularMarketChangePercent": -1.01178,
        "exchange": "NYQ",
        "messageBoardId": "finmb_160233",
        "exchangeTimezoneName": "America/New_York",
        "exchangeTimezoneShortName": "EST",
        "gmtOffSetMilliseconds": -18000000,
        "market": "us_market",
        "esgPopulated": false,
        "corporateActions": [],
        "postMarketTime": 1740613200,
        "regularMark